<p style="font-size:36px;text-align:center"> <b>MOVIE RECOMMENDER SYSTEM</b> </p>

##### Problem Statement:
- Recommend the movies based on the movie he/she has liked.

In [74]:
# importing the required libraries

import numpy as np
import pandas as pd
import ast

import nltk
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings('ignore')

-----------------------

### DATA OVERVIEW ::

In [2]:
movies=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')

movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [3]:
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [4]:
print(movies.shape)
print(credits.shape)

(4803, 20)
(4803, 4)


In [5]:
movies.iloc[1]

budget                                                          300000000
genres                  [{"id": 12, "name": "Adventure"}, {"id": 14, "...
homepage                     http://disney.go.com/disneypictures/pirates/
id                                                                    285
keywords                [{"id": 270, "name": "ocean"}, {"id": 726, "na...
original_language                                                      en
original_title                   Pirates of the Caribbean: At World's End
overview                Captain Barbossa, long believed to be dead, ha...
popularity                                                        139.083
production_companies    [{"name": "Walt Disney Pictures", "id": 2}, {"...
production_countries    [{"iso_3166_1": "US", "name": "United States o...
release_date                                                   2007-05-19
revenue                                                         961000000
runtime                               

In [6]:
credits.cast.iloc[0]

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [7]:
# Merging the two data sets

movies=movies.merge(credits,on='title')

movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


----------

- Need to create a new data frame where we will be having movie id-title-tags(combining other colummns)
- By using tags we can find the similarity between the movies and recommend the movies with closest similarity

----------------

### DATA CLEANING AND PREPROCESSING ::

In [9]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')

In [11]:
# keeping only the required columns
movies= movies[['movie_id','title','overview','genres','keywords', 'cast', 'crew']]

movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [12]:
# Checking for null values
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [13]:
# Since no much missing values we can drop those 3 rows

movies.dropna(inplace=True)

In [14]:
# Checking for duplicates

movies.duplicated().sum()

0

In [15]:
# Having a check at columns to bring them into correct format we need

movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [18]:
movies.iloc[0].keywords

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [16]:
#it is a list of dictionary and its a string......we need them in ['Action','Adventure','Fantasy','Science Fiction']

k='[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'
print(k)
ast.literal_eval(k)

[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]


[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [17]:
def convert(obj):
    l=[]
    
    for i in ast.literal_eval(obj):
        l.append(i['name'])
        
    return l

movies['genres'].apply(convert)

0       [Action, Adventure, Fantasy, Science Fiction]
1                        [Adventure, Fantasy, Action]
2                          [Action, Adventure, Crime]
3                    [Action, Crime, Drama, Thriller]
4                [Action, Adventure, Science Fiction]
                            ...                      
4804                        [Action, Crime, Thriller]
4805                                [Comedy, Romance]
4806               [Comedy, Drama, Romance, TV Movie]
4807                                               []
4808                                    [Documentary]
Name: genres, Length: 4806, dtype: object

In [19]:
movies['genres']= movies['genres'].apply(convert)
movies['keywords']=movies['keywords'].apply(convert)

movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [20]:
movies['cast'][0]  # Here I need the name of the 3 main actosr/actresses

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [21]:
def convert2(obj):
    l=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter !=3:    
            l.append(i['name'])
            counter+=1
        else:
            break
        
    return l

movies['cast'].apply(convert2)

0        [Sam Worthington, Zoe Saldana, Sigourney Weaver]
1           [Johnny Depp, Orlando Bloom, Keira Knightley]
2            [Daniel Craig, Christoph Waltz, Léa Seydoux]
3            [Christian Bale, Michael Caine, Gary Oldman]
4          [Taylor Kitsch, Lynn Collins, Samantha Morton]
                              ...                        
4804    [Carlos Gallardo, Jaime de Hoyos, Peter Marqua...
4805         [Edward Burns, Kerry Bishé, Marsha Dietlein]
4806           [Eric Mabius, Kristin Booth, Crystal Lowe]
4807            [Daniel Henney, Eliza Coupe, Bill Paxton]
4808    [Drew Barrymore, Brian Herzlinger, Corey Feldman]
Name: cast, Length: 4806, dtype: object

In [22]:
movies['cast']=movies['cast'].apply(convert2)

In [23]:
movies['crew'][0] # needed the director name alone

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [24]:
def fetch_director(obj):
    l=[]
    
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            l.append(i['name'])
            break
        
    return l

movies['crew']=movies['crew'].apply(fetch_director)

In [25]:
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]


In [26]:
movies['overview'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [27]:
#its a string we will comvert it to list so that we can combine things

movies['overview']=movies['overview'].apply(lambda x:x.split())

In [28]:
movies.head(2)   # final data we got

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]


------------------------------------------------------------------------------------

- Everything is in a list we will combine them and convert them into a string
- we need to remove spaces between lists because it can create two different tags and that could effect our model

----------------

In [30]:
movies['genres'].apply(lambda x:[i.replace(' ','') for i in x])

0       [Action, Adventure, Fantasy, ScienceFiction]
1                       [Adventure, Fantasy, Action]
2                         [Action, Adventure, Crime]
3                   [Action, Crime, Drama, Thriller]
4                [Action, Adventure, ScienceFiction]
                            ...                     
4804                       [Action, Crime, Thriller]
4805                               [Comedy, Romance]
4806               [Comedy, Drama, Romance, TVMovie]
4807                                              []
4808                                   [Documentary]
Name: genres, Length: 4806, dtype: object

In [31]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(' ','') for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(' ','') for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(' ','') for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(' ','') for i in x])

In [32]:
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]


In [33]:
# create a new column tags

movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [34]:
print(movies['tags'][0],end=' ')

['In', 'the', '22nd', 'century,', 'a', 'paraplegic', 'Marine', 'is', 'dispatched', 'to', 'the', 'moon', 'Pandora', 'on', 'a', 'unique', 'mission,', 'but', 'becomes', 'torn', 'between', 'following', 'orders', 'and', 'protecting', 'an', 'alien', 'civilization.', 'Action', 'Adventure', 'Fantasy', 'ScienceFiction', 'cultureclash', 'future', 'spacewar', 'spacecolony', 'society', 'spacetravel', 'futuristic', 'romance', 'space', 'alien', 'tribe', 'alienplanet', 'cgi', 'marine', 'soldier', 'battle', 'loveaffair', 'antiwar', 'powerrelations', 'mindandsoul', '3d', 'SamWorthington', 'ZoeSaldana', 'SigourneyWeaver', 'JamesCameron'] 

In [35]:
# droping other columns
new_df=movies[['movie_id','title','tags']]
new_df.head(2)

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."


In [37]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

In [38]:
new_df['tags'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'

In [39]:
# Converting to lower case
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

- performing text vectorizations...on tags
- converting them to vectors to have a check at similarity
- Initially picking 5000 words to see similarity

In [41]:
cv=CountVectorizer(max_features=5000, stop_words='english')
tf=TfidfVectorizer(max_features=5000, stop_words='english')

In [42]:
cv.fit_transform(new_df['tags']).toarray()
tf.fit_transform(new_df['tags']).toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [43]:
print(cv.fit_transform(new_df['tags']).toarray().shape)
print(tf.fit_transform(new_df['tags']).toarray().shape )

(4806, 5000)
(4806, 5000)


In [44]:
vectors=cv.fit_transform(new_df['tags']).toarray() 
vectors2=tf.fit_transform(new_df['tags']).toarray() 

In [45]:
cv.get_feature_names()[100:110]

['adultanimation',
 'adultery',
 'adulthood',
 'adults',
 'advantage',
 'adventure',
 'adventures',
 'advertising',
 'advice',
 'affair']

In [47]:
# performing stemming

ps=PorterStemmer()

def stem(text):
    
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
        
    return str(" ").join(y)

new_df['tags']=new_df['tags'].apply(stem)

In [48]:
cv=CountVectorizer(max_features=5000, stop_words='english')
vectors=cv.fit_transform(new_df['tags']).toarray() 

In [49]:
tf=TfidfVectorizer(max_features=5000, stop_words='english')
vectors2=tf.fit_transform(new_df['tags']).toarray() 

In [50]:
cv.get_feature_names()[100:110]

['adrienbrodi',
 'adult',
 'adultanim',
 'adulteri',
 'adulthood',
 'advanc',
 'adventur',
 'adventure',
 'adventures',
 'advertis']

---------------------

### MODEL BUILDING ::

- Need to compute distance between movies using cosine distance/Cosine Similarity as we have higher dimensions Euclidean distance will not work well 

In [51]:
cosine_similarity(vectors) 
cosine_similarity(vectors2) 

array([[1.        , 0.02203984, 0.03038546, ..., 0.02319866, 0.        ,
        0.        ],
       [0.02203984, 1.        , 0.0129118 , ..., 0.01700865, 0.        ,
        0.00642518],
       [0.03038546, 0.0129118 , 1.        , ..., 0.01628553, 0.        ,
        0.        ],
       ...,
       [0.02319866, 0.01700865, 0.01628553, ..., 1.        , 0.01860761,
        0.02850017],
       [0.        , 0.        , 0.        , ..., 0.01860761, 1.        ,
        0.0197017 ],
       [0.        , 0.00642518, 0.        , ..., 0.02850017, 0.0197017 ,
        1.        ]])

In [52]:
similarity=cosine_similarity(vectors)
similarity2=cosine_similarity(vectors2)

In [53]:
similarity.shape

(4806, 4806)

In [54]:
similarity[0]

array([1.        , 0.08346223, 0.0860309 , ..., 0.04499213, 0.        ,
       0.        ])

In [55]:
new_df[new_df['title']=='Avatar'].index[0]

0

In [56]:
# Sorting the movies

sorted(similarity[10],reverse=True) 

[1.0,
 0.41962716370814124,
 0.40522160042937677,
 0.38464375683498514,
 0.3839908587347318,
 0.3525119939553163,
 0.34415494032603927,
 0.325,
 0.3228859228101098,
 0.3169328455231938,
 0.3071475584169756,
 0.30588049014524776,
 0.29698484809835,
 0.29294300271021917,
 0.2908872369413698,
 0.2834217155626206,
 0.2814699843903005,
 0.2795084971874737,
 0.2795084971874737,
 0.2790781528257191,
 0.27892960426948415,
 0.27570548328517797,
 0.275391241934913,
 0.27386127875258315,
 0.2729185521038726,
 0.2727977357881894,
 0.2711630722733202,
 0.26843774609657967,
 0.26837252006084666,
 0.2677338527889283,
 0.26424909878161656,
 0.26352313834736496,
 0.2631773073081477,
 0.2631773073081477,
 0.2631174057921088,
 0.26311740579210874,
 0.2629502940535666,
 0.26233033431358116,
 0.26220221204253796,
 0.2622022120425379,
 0.2622022120425379,
 0.2620001293828119,
 0.25952022323637275,
 0.25819888974716115,
 0.25649458802128855,
 0.2564388222660863,
 0.25558282541117916,
 0.25495097567963926,
 0

In [57]:
list(enumerate(similarity[10]))

[(0, 0.10259783520851541),
 (1, 0.10846522890932808),
 (2, 0.08385254915624211),
 (3, 0.08951435925492911),
 (4, 0.20756952180649915),
 (5, 0.23241742005034208),
 (6, 0.019611613513818404),
 (7, 0.26220221204253796),
 (8, 0.17320508075688773),
 (9, 0.23570226039551584),
 (10, 1.0),
 (11, 0.18450624160577703),
 (12, 0.14680505487867584),
 (13, 0.0657793514480272),
 (14, 0.325),
 (15, 0.14288690166235204),
 (16, 0.12824729401064427),
 (17, 0.1556997888323046),
 (18, 0.22206996305928156),
 (19, 0.0607332247634187),
 (20, 0.29698484809835),
 (21, 0.21380899352993948),
 (22, 0.09682458365518543),
 (23, 0.08519427513705972),
 (24, 0.10397504898200727),
 (25, 0.06629935441317959),
 (26, 0.175),
 (27, 0.1619552660357832),
 (28, 0.1419904585617662),
 (29, 0.04225771273642583),
 (30, 0.23890473351968813),
 (31, 0.15504341823651058),
 (32, 0.08304547985373996),
 (33, 0.1264911064067352),
 (34, 0.0),
 (35, 0.16137430609197573),
 (36, 0.19289712886816485),
 (37, 0.051298917604257706),
 (38, 0.21740

In [59]:
# sorting based on index values
sorted(list(enumerate(similarity[10])),reverse=True, key=lambda x:x[1])

[(10, 1.0),
 (870, 0.41962716370814124),
 (1299, 0.40522160042937677),
 (2439, 0.38464375683498514),
 (813, 0.3839908587347318),
 (232, 0.3525119939553163),
 (79, 0.34415494032603927),
 (14, 0.325),
 (3611, 0.3228859228101098),
 (3093, 0.3169328455231938),
 (193, 0.3071475584169756),
 (4529, 0.30588049014524776),
 (20, 0.29698484809835),
 (3228, 0.29294300271021917),
 (1143, 0.2908872369413698),
 (4453, 0.2834217155626206),
 (1322, 0.2814699843903005),
 (1363, 0.2795084971874737),
 (2433, 0.2795084971874737),
 (1251, 0.2790781528257191),
 (4478, 0.27892960426948415),
 (583, 0.27570548328517797),
 (4448, 0.275391241934913),
 (182, 0.27386127875258315),
 (1992, 0.2729185521038726),
 (4383, 0.2727977357881894),
 (203, 0.2711630722733202),
 (3832, 0.26843774609657967),
 (1903, 0.26837252006084666),
 (1120, 0.2677338527889283),
 (664, 0.26424909878161656),
 (1362, 0.26352313834736496),
 (1202, 0.2631773073081477),
 (2867, 0.2631773073081477),
 (2856, 0.2631174057921088),
 (4451, 0.263117405

In [60]:
# For top 5 recommendations
sorted(list(enumerate(similarity[10])),reverse=True, key=lambda x:x[1])[1:6]

[(870, 0.41962716370814124),
 (1299, 0.40522160042937677),
 (2439, 0.38464375683498514),
 (813, 0.3839908587347318),
 (232, 0.3525119939553163)]

----------------------------

#### RECOMMENDER-1 

In [71]:
# recommendation based on Bag of words

def recommend(movie):
    
    movie_index=new_df[new_df['title']==movie].index[0]
    distances=similarity[movie_index]
    movie_list=sorted(list(enumerate(distances)),reverse=True, key=lambda x:x[1])[1:6]
    
    for i in movie_list:  
        print(new_df.title.iloc[i[0]])

In [72]:
recommend("Mission: Impossible")

Mission: Impossible II
Dying of the Light
Mission: Impossible III
The November Man
Tinker Tailor Soldier Spy


In [73]:
recommend('Batman Begins')

The Dark Knight
Batman
Batman
The Dark Knight Rises
10th & Wolf


-----------------------------------

#### RECOMMENDER-2

In [68]:
# recommendation based on TfIdf  vectorizer

def recommend2(movie):
    
    movie_index=new_df[new_df['title']==movie].index[0]
    distances=similarity2[movie_index]
    movie_list=sorted(list(enumerate(distances)),reverse=True, key=lambda x:x[1])[1:6]
    
    for i in movie_list:  
        print(new_df.title.iloc[i[0]])

In [69]:
recommend2('Batman Begins')

The Dark Knight
The Dark Knight Rises
Batman
Batman v Superman: Dawn of Justice
Batman Returns


In [70]:
recommend2("Mission: Impossible")

Mission: Impossible II
Mission: Impossible III
Dying of the Light
Tinker Tailor Soldier Spy
The November Man


In [ ]:
# Tfidf works well....

# As a part of analysis we can further increase the features and can have a check how our vectors perform

--------------------------------------------

In [65]:
#pickle.dump(new_df,open('movies.pkl','wb'))
#pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))
#pickle.dump(similarity,open("similarity.pkl",'wb'))